In [0]:
prompt_id = 2
model_name = "our+overall"
    # "our", "our+sent", "our+overall", "our+sent+overall"

In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()

  print('TPU devices:')
  pprint.pprint(devices)

In [0]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/AES謝"
os.chdir(path)
# os.listdir(path)

In [0]:
import argparse
import nltk, keras, json
import tensorflow as tf
from keras.models import Model
from keras import layers
from keras.layers import Input, concatenate, Embedding, LSTM, Dense, Lambda
from keras.engine.topology import Layer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

nltk.download('stopwords')
nltk.download('punkt')
# stopwords, punkt, 

In [0]:
# Mean Over Time
class TemporalMean(Layer):
  def __init__(self, **kwargs):
      self.supports_masking = True
      super(TemporalMean, self).__init__(**kwargs)

  def compute_mask(self, input, input_mask=None):
      # do not pass the mask to the next layers
      return None

  def call(self, x, mask=None):
      if mask is not None:
          # mask (batch, time)
          mask = K.cast(mask, K.floatx())
          # mask (batch, x_dim, time)
          mask = K.repeat(mask, x.shape[-1])
          # mask (batch, time, x_dim)
          mask = tf.transpose(mask, [0,2,1])
          x = x * mask
      return K.sum(x, axis=1) / K.sum(mask, axis=1)

  def compute_output_shape(self, input_shape):
      # remove temporal dimension
      return (input_shape[0], input_shape[2])

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def get_clean_essays(Text):
    clean_essays = []
    for essay in Text:
        clean_essays.append(essay_to_wordlist(essay, remove_stopwords=False))
    return clean_essays

def create_dict(text):
    print("create_dict")
    word_index = {}
    word_index['xieyikuan'] = 0
    for sentence in text:
        for word in sentence:
            if word not in word_index:
                word_index[word] = len(word_index)
    word_index['unk'] = len(word_index)
    return word_index

def essay_to_index(essays, word_index):
    Essays = []
    for essay in essays:
        sentence = []
        for sent in essay:
            words = []
            for word in sent:
                words.append(word_index[word])
                '''
                if word in word_index:
                    words.append(word_index[word])
                elif word not in word_index:
                    words.append(word_index['unk'])
                '''
            sentence.append(words)
        Essays.append(sentence)
    return Essays

def get_embeddings_index():
    embeddings_index = {}
    f = open('glove.6B.50d.txt', 'r', encoding='UTF-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

def embeddings_matrix(embeddings_index, word_index, embedding_dim):
    nb_words = len(word_index)
    embedding_matrix = np.zeros((nb_words + 1, embedding_dim))
    for word, i in word_index.items():
        if i > nb_words:
            continue
        embeddings_vector = embeddings_index.get(word)
        if embeddings_vector is not None:
            embedding_matrix[i] = embeddings_vector
        if embeddings_vector is None:
            embedding_matrix[i] = embeddings_index.get('unk')
    return embedding_matrix

def get_pst_feature(essays):
    pst_feature = []
    for essay in essays:
        position = []
        i = 1
        for sent in essay:
            position.append(max(1 / i, 1 / (len(essay) - i + 1)))
            i += 1
        position = np.array(position).reshape(1, -1)
        pst_feature.append(position)
    return pst_feature

def stack(tensor):
    Tensor = keras.backend.stack(tensor, axis=1)
    return Tensor

# --------
csv_input = pd.read_csv(filepath_or_buffer="data_feature/set"+str(prompt_id)+"_feature(26).tsv", delimiter="\t", engine="python", error_bad_lines=False)

Text = csv_input['essay']
Score = csv_input['domain1_score']
Label = csv_input['domain1_score']

# --------
if prompt_id == 1:
    Score = (Score - 2) / 10        # set_1     范围 2-12
    max_sent_length, max_word_length = 40, 35
elif prompt_id == 2:
    Score = (Score - 1) / 5         # set_2     范围 1-6
    max_sent_length, max_word_length = 40, 40
elif prompt_id == 3:
    Score = Score / 3               # set_3     范围 0-3
    max_sent_length, max_word_length = 15, 40
elif prompt_id == 4:
    Score = Score / 3               # set_4     范围 0-3
    max_sent_length, max_word_length = 10, 50
elif prompt_id == 5:
    Score = Score / 4               # set_5     范围 0-4
    max_sent_length, max_word_length = 15, 40
elif prompt_id == 6:
    Score = Score / 4               # set_6     范围 0-4
    max_sent_length, max_word_length = 15, 40
elif prompt_id == 7:
    Score = Score / 30              # set_7     范围 0-30
    max_sent_length, max_word_length = 27, 35
elif prompt_id == 8:
    Score = Score / 60              # set_8     范围 0-60
    max_sent_length, max_word_length = 65, 37


# Get Overall Feature (25)
if model_name in ["our+overall", "our+overall_mot", "our+sent+overall", "our+sent+overall_mot", "our+all", "our+all_mot"]:
    overall_feature = csv_input[['word_count', 'avg_word_len', 'sentence_count', 'avg_sentence_len', 'lemma_count', 'noun_count', 'adj_count', 'adv_count', 'verb_count', 'cc_count',
                                 'syllable_count', 'flesch_reading_ease', 'smog_index', 'flesch_kincaid_grade', 'coleman_liau_index', 'automated_readability_index',
                                 'dale_chall_readability_score', 'difficult_words', 'linsear_write_formula',
                                 'gunning_fog', 'spelling_errors', 'exc_count', 'que_count', 'comma_count', 'stopwords_num']]
    # overall_feature = np.array(np.sqrt(overall_feature))
    overall_feature = np.array(overall_feature)
    num_feature = 25
    where_are_nan = np.isnan(overall_feature)
    where_are_inf = np.isinf(overall_feature)
    overall_feature[where_are_nan] = 0
    overall_feature[where_are_inf] = 0
    # standardized
    stdsc = StandardScaler()
    overall_feature = stdsc.fit_transform(overall_feature)

# Get the Second Last Layer of BERT
if model_name in ["our+bert", "our+bert_mot", "our+all", "our+all_mot"]:
    bert = pd.read_csv(filepath_or_buffer="fine_tuning_out/essay_set_"+str(prompt_id)+"_output.tsv", delimiter="\t", engine="python", error_bad_lines=False)
    bert = np.array(bert)
    max_seq_length = len(bert[0])

# 做字典
clean_essays = get_clean_essays(Text)
word_index = create_dict(clean_essays)

# 做分句
text = np.array(Text)
a = []
for essay in text:
    a.append(sent_tokenize(essay))
texts = np.array(a)

# 做分词
essays = []
for essay in texts:
    sentence = []
    for sents in essay:
        sent = essay_to_wordlist(sents, remove_stopwords=True)
        # sent = np.array(sent)
        sentence.append(sent)
    essays.append(sentence)
essays = np.array(essays)

Essays = essay_to_index(essays, word_index)
Essays = np.array(Essays)

# Get Sentence Feature
if model_name in ["our+sent", "our+sent_mot", "our+sent+overall", "our+sent+overall_mot", "our+all""our+all_mot"]:
    f = open('sent_feature/set'+str(prompt_id)+'_sent.json', 'r+')
    str_json = f.read()
    temp = json.loads(str_json)
    essay_num = len(temp)
    sent_num = max_sent_length
    sent_features = []
    for i in range(len(temp)):
        essay = temp['essay'+str(i)]
        feature_sent2essay = []
        for j in range(sent_num):
            sent = essay['sent'+str(j)]
            feature_sent = []
            feature_sent.append(sent['word_count'])
            feature_sent.append(sent['avg_word_len'])
            feature_sent.append(sent['lemma_count'])
            feature_sent.append(sent['noun_count'])
            feature_sent.append(sent['adj_count'])
            feature_sent.append(sent['adv_count'])
            feature_sent.append(sent['verb_count'])
            feature_sent.append(sent['cc_count'])
            feature_sent.append(sent['syllable_count'])
            feature_sent.append(sent['flesch_reading_ease'])
            feature_sent.append(sent['flesch_kincaid_grade'])
            feature_sent.append(sent['coleman_liau_index'])
            feature_sent.append(sent['automated_readability_index'])
            feature_sent.append(sent['dale_chall_readability_score'])
            feature_sent.append(sent['difficult_words'])
            feature_sent.append(sent['linsear_write_formula'])
            feature_sent.append(sent['gunning_fog'])
            feature_sent.append(sent['spelling_errors'])
            feature_sent.append(sent['stopwords_num'])
            feature_sent.append(sent['comma_count'])
            feature_sent2essay.append(feature_sent)
        sent_features.append(feature_sent2essay)
    sent_features = np.array(sent_features)

    pst_feature = get_pst_feature(essays)
    pad_pst_feature = []
    for feature in pst_feature:
        pad_feature = pad_sequences(feature, maxlen=max_sent_length, dtype='float32')
        pad_pst_feature.append(pad_feature)
    position_feature = np.array(pad_pst_feature)
    position_feature = np.array([mat.T for mat in position_feature])

    sent_features = np.concatenate((position_feature, sent_features), axis=-1)
    num_sent_feature = 21

    stdsc = StandardScaler()
    Sent_features = []
    for essay in sent_features:
        Sent_features.append(stdsc.fit_transform(essay))
    sent_features = np.array(Sent_features)

# Padding Words and Sentences
essay_pad = []
for essay in Essays:
    pad = pad_sequences(essay, maxlen=max_word_length)
    essay_pad.append(pad)
essays_pad = []
for essay in essay_pad:
    essays = []
    for sent in essay:
        for word in sent:
            essays.append(word)
    essays_pad.append(essays)
essays_pad = np.array(essays_pad)
Padding_Length = max_word_length * max_sent_length
essays_pad = pad_sequences(essays_pad, maxlen=Padding_Length)

# Get Embedding Matrix
embeddings_index = get_embeddings_index()
embeddings_matrix = embeddings_matrix(embeddings_index, word_index, embedding_dim=50)

# 记录成绩
results = []
score_predict_list = []
sssscore = []
llllabel = []
newresults = []
cv = KFold(n_splits=5, shuffle=True, random_state=10)
count = 0
for train, test in cv.split(essays_pad):
    text_train, text_test = essays_pad[train], essays_pad[test]
    label_train, label_test = Score.iloc[train], Score.iloc[test]
    score_train, score_test = Label.iloc[train], Label.iloc[test]
    if model_name in ["our+overall", "our+overall_mot", "our+sent+overall", "our+sent+overall_mot", "our+all", "our+all_mot"]:
        overall_train, overall_test = overall_feature[train], overall_feature[test]
    if model_name in ["our+sent", "our+sent_mot", "our+sent+overall", "our+sent+overall_mot", "our+all""our+all_mot"]:
        sent_train, sent_test = sent_features[train], sent_features[test]
    if model_name in ["our+bert", "our+bert_mot", "our+all", "our+all_mot"]:
        bert_train, bert_test = bert[train], bert[test]

    print('Build model '+str(count)+'...')
    count += 1

    # -------
    text_input = Input(shape=(Padding_Length, ), name='text_input')
    if model_name in ["our+overall", "our+overall_mot", "our+sent+overall", "our+sent+overall_mot", "our+all", "our+all_mot"]:
        overall_feature_input = Input(shape=(num_feature,), name='overall_feature_input')
    if model_name in ["our+sent", "our+sent_mot", "our+sent+overall", "our+sent+overall_mot", "our+all""our+all_mot"]:
        sentence_feature_input = Input(shape=(max_sent_length, num_sent_feature,), name='sentence_feature_input')
    if model_name in ["our+bert", "our+bert_mot", "our+all", "our+all_mot"]:
        bert_input = Input(shape=(max_seq_length,), name='bert_input')

    embedding_layer = Embedding(input_dim=len(word_index) + 1,
                                output_dim=50,
                                weights=[embeddings_matrix],
                                input_length=Padding_Length,
                                trainable=True,
                                mask_zero=True)
    embedding = embedding_layer(text_input)
    if model_name == "our":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=False)(sent_output)
        dropout = layers.Dropout(0.5)(lstm2)
        output = Dense(1, activation='sigmoid')(dropout)
        model = Model(inputs=text_input, outputs=output)

    elif model_name == "our_mot":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(sent_output)
        mot = TemporalMean()(lstm2)
        dropout = layers.Dropout(0.5)(mot)
        output = Dense(1, activation='sigmoid')(dropout)
        model = Model(inputs=text_input, outputs=output)

    elif model_name == "our+sent":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)
        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)
        # 每个句子最后的输出结合句子的特征值
        sent_feature_concat_layer = Lambda(concatenate)([sent_output, sentence_feature_input])
        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=False)(sent_feature_concat_layer)
        dropout = layers.Dropout(0.5)(lstm2)
        output = Dense(1, activation='sigmoid')(dropout)
        model = Model(inputs=[text_input, sentence_feature_input], outputs=output)

    elif model_name == "our+sent_mot":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)
        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)
        # 每个句子最后的输出结合句子的特征值
        sent_feature_concat_layer = Lambda(concatenate)([sent_output, sentence_feature_input])
        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(sent_feature_concat_layer)
        mot = TemporalMean()(lstm2)
        dropout = layers.Dropout(0.5)(mot)
        output = Dense(1, activation='sigmoid')(dropout)
        model = Model(inputs=[text_input, sentence_feature_input], outputs=output)

    elif model_name == "our+overall":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=False)(sent_output)
        dropout = layers.Dropout(0.5)(lstm2)

        # LSTM层输出结合全体特征和BERT
        concat_layer = Lambda(concatenate)([dropout, overall_feature_input])

        output = Dense(1, activation='sigmoid')(concat_layer)
        model = Model(inputs=[text_input, overall_feature_input], outputs=output)

    elif model_name == "our+overall_mot":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(sent_output)
        mot = TemporalMean()(lstm2)
        dropout = layers.Dropout(0.5)(mot)

        # LSTM层输出结合全体特征和BERT
        concat_layer = Lambda(concatenate)([dropout, overall_feature_input])

        output = Dense(1, activation='sigmoid')(concat_layer)
        model = Model(inputs=[text_input, overall_feature_input], outputs=output)

    elif model_name == "our+bert":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=False)(sent_output)
        dropout = layers.Dropout(0.5)(lstm2)

        # LSTM层输出结合全体特征和BERT
        concat_layer = Lambda(concatenate)([dropout, bert_input])

        output = Dense(1, activation='sigmoid')(concat_layer)
        model = Model(inputs=[text_input, bert_input], outputs=output)

    elif model_name == "our+bert_mot":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        # 将每个句子的最后输出重新组合
        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(sent_output)
        mot = TemporalMean()(lstm2)
        dropout = layers.Dropout(0.5)(mot)

        # LSTM层输出结合全体特征和BERT
        concat_layer = Lambda(concatenate)([dropout, bert_input])

        output = Dense(1, activation='sigmoid')(concat_layer)
        model = Model(inputs=[text_input, bert_input], outputs=output)

    elif model_name == "our+sent+overall":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        sent_feature_concat_layer = Lambda(concatenate)([sent_output, sentence_feature_input])
        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=False)(sent_feature_concat_layer)
        dropout = layers.Dropout(0.5)(lstm2)
        concat_layer = Lambda(concatenate)([dropout, overall_feature_input])
        output = Dense(1, activation='sigmoid')(concat_layer)
        model = Model(inputs=[text_input, overall_feature_input, sentence_feature_input], outputs=output)

    elif model_name == "our+sent+overall_mot":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        sent_feature_concat_layer = Lambda(concatenate)([sent_output, sentence_feature_input])
        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(sent_feature_concat_layer)
        mot = TemporalMean()(lstm2)
        dropout = layers.Dropout(0.5)(mot)
        concat_layer = Lambda(concatenate)([dropout, overall_feature_input])
        output = Dense(1, activation='sigmoid')(concat_layer)
        model = Model(inputs=[text_input, overall_feature_input, sentence_feature_input], outputs=output)

    elif model_name == "our+all":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        sent_feature_concat_layer = Lambda(concatenate)([sent_output, sentence_feature_input])
        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=False)(sent_feature_concat_layer)
        dropout = layers.Dropout(0.5)(lstm2)
        concat_layer = Lambda(concatenate)([dropout, overall_feature_input, bert_input])
        output = Dense(1, activation='sigmoid')(concat_layer)

        model = Model(inputs=[text_input, overall_feature_input, sentence_feature_input, bert_input], outputs=output)

    elif model_name == "our+all_mot":
        lstm1 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(embedding)

        a = [i for i in range(max_word_length - 1, max_word_length * max_sent_length, max_word_length)]
        sent_layer = [Lambda(lambda t: t[:, i, :])(lstm1) for i in a]
        sent_output = Lambda(stack)(sent_layer)

        sent_feature_concat_layer = Lambda(concatenate)([sent_output, sentence_feature_input])
        lstm2 = LSTM(300, dropout=0.5, recurrent_dropout=0.1, return_sequences=True)(sent_feature_concat_layer)
        mot = TemporalMean()(lstm2)
        dropout = layers.Dropout(0.5)(mot)
        concat_layer = Lambda(concatenate)([dropout, overall_feature_input])
        output = Dense(1, activation='sigmoid')(concat_layer)
        model = Model(inputs=[text_input, overall_feature_input, sentence_feature_input, bert_input], outputs=output)

    # -------

    model.summary()

    model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001, decay=1e-6), metrics=['mean_absolute_error'])

    if model_name in ["our", "our_mot"]:
        model.fit(text_train, label_train, batch_size=32, epochs=50, verbose=2)
    elif model_name in ["our+sent", "our+sent_mot"]:
        model.fit([text_train, sent_train], label_train, batch_size=32, epochs=50, verbose=2)
    elif model_name in ["our+overall", "our+overall_mot"]:
        model.fit([text_train, overall_train], label_train, batch_size=32, epochs=50, verbose=2)
    elif model_name in ["our+bert", "our+bert_mot"]:
        model.fit([text_train, bert_train], label_train, batch_size=32, epochs=50, verbose=2)
    elif model_name in ["our+sent+overall", "our+sent+overall_mot"]:
        model.fit([text_train, overall_train, sent_train], label_train, batch_size=32, epochs=50, verbose=2)
    elif model_name in ["our+all", "our+all_mot"]:
        model.fit([text_train, overall_train, sent_train, bert_train], label_train, batch_size=32, epochs=50, verbose=2)

    if model_name in ["our", "our_mot"]:
        score_predict = model.predict(text_test)
    elif model_name in ["our+sent", "our+sent_mot"]:
        score_predict = model.predict([text_test, sent_test])
    elif model_name in ["our+overall", "our+overall_mot"]:
        score_predict = model.predict([text_test, overall_test])
    elif model_name in ["our+bert", "our+bert_mot"]:
        score_predict = model.predict([text_test, bert_test])
    elif model_name in ["our+sent+overall", "our+sent+overall_mot"]:
        score_predict = model.predict([text_test, overall_test, sent_test])
    elif model_name in ["our+all", "our+all_mot"]:
        score_predict = model.predict([text_test, overall_test, sent_test, bert_test])

    if prompt_id == 1:
        score_predict = score_predict*10+2
    elif prompt_id == 2:
        score_predict = score_predict*5+1
    elif prompt_id == 3 or prompt_id == 4:
        score_predict = score_predict*3
    elif prompt_id == 5 or prompt_id == 6:
        score_predict = score_predict*4
    elif prompt_id == 7:
        score_predict = score_predict*30
    elif prompt_id == 8:
        score_predict = score_predict*60

    score_predict = np.around(score_predict)

    sssscore.append(score_predict)
    llllabel.append(score_test)

    result = cohen_kappa_score(score_test.values, score_predict, weights='quadratic')
    print("Kappa Score: {}".format(result))
    results.append(result)
    for i in results:
      if i != 0:
        newresults.append(i)

print("Average Kappa score after a 5-fold cross validation: ", np.around(np.array(newresults).mean(), decimals=5))

sssscore = np.array(sssscore)
res = []
for scores in sssscore:
  for score in scores:
    res.append(score)

ll = np.array(llllabel)
res_label = []
for labels in ll:
  for label in labels:
    res_label.append(label)
res_label = np.array(res_label)

QWK = cohen_kappa_score(res_label, res, weights='quadratic')
print("Kappa score after a 5-fold cross validation: ", QWK)